In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from tqdm import tqdm_notebook
import string
from collections import Counter
import itertools
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report
import pickle
from tqdm.auto import tqdm
tqdm.pandas()
## pip install swifter #for parallelization
import swifter

## Initial Model

In [4]:
ca_restaurant_reviews=pd.read_csv("california_restaurants.csv")
ca_restaurant_reviews=ca_restaurant_reviews.reset_index(drop = True)

In [12]:
# texts = []
# stars = [ca_restaurant_reviews['review_stars'] for review in ca_restaurant_reviews]
# pbar = tqdm(total=ca_restaurant_reviews.shape[0]+1)
# for index, row in ca_restaurant_reviews.iterrows():
#     texts.append(clean_text(row['text']))
#     pbar.update(1)
# pbar.close()

100%|█████████▉| 205464/205465 [01:08<00:00, 2980.38it/s]


In [13]:
ca_restaurant_reviews.head(1)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,2456,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,1,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': ""...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...",bFPdtzu11Oi0f92EAcjqmg,5,What a great addition to the Funk Zone! Grab ...,2016-10-13 22:50:47


pandas.core.series.Series

In [14]:
texts = [ca_restaurant_reviews['text'] for review in ca_restaurant_reviews]

In [40]:
new_text=ca_restaurant_reviews['text'].values.tolist()
new_stars=ca_restaurant_reviews["review_stars"].values.tolist()

In [44]:
X_train, X_test, y_train, y_test = train_test_split(new_text, new_stars, test_size=0.15, random_state=42, shuffle =False)

In [50]:
training_pipe=Pipeline([("Vectorization",TfidfVectorizer()),("SVC",LinearSVC())])

In [51]:
training_pipe

Pipeline(steps=[('Vectorization', TfidfVectorizer()), ('SVC', LinearSVC())])

In [48]:
y_train[0]

5

In [52]:
training_pipe.fit(X_train,y_train)

Pipeline(steps=[('Vectorization', TfidfVectorizer()), ('SVC', LinearSVC())])

In [67]:
out=training_pipe.predict(['Worst Restaurant'])[0]

In [68]:
out

1

In [ ]:
with open('svcpipe.pickle', 'wb') as f:
  pickle.dump(training_pipe, f)

In [75]:

with open('svcpipe.pickle', 'rb') as f:
  loadedpipe = pickle.load(f)
testing=loadedpipe.predict(["This is a good restaurant, but the service is average"])
testing[0]

3

In [84]:
print('Training set score: ' + str(loadedpipe.score(X_train,y_train)))
print('Test set score: ' + str(loadedpipe.score(X_test,y_test)))


Training set score: 0.7952177000068711
Test set score: 0.6995133030499675


### SVC

In [79]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [81]:
newpipeline = Pipeline([('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()), 
    ('clf', LinearSVC())])

In [82]:
newpipeline.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC())])

In [83]:
print('Training set score: ' + str(newpipeline.score(X_train,y_train)))
print('Test set score: ' + str(newpipeline.score(X_test,y_test)))

Training set score: 0.7952177000068711
Test set score: 0.6995133030499675


### NaiveBayes model

In [85]:
naivebayespipe = Pipeline([('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()), 
    ('clf', MultinomialNB())])

In [86]:
naivebayespipe.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [87]:
print('Training set score: ' + str(naivebayespipe.score(X_train,y_train)))
print('Test set score: ' + str(naivebayespipe.score(X_test,y_test)))

Training set score: 0.5564634341861158
Test set score: 0.5980856586632057


### SGD Classifier

In [94]:
from sklearn.linear_model import SGDClassifier

svmpipe = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf-svm', SGDClassifier(loss='modified_huber', penalty='l1',alpha=1e-3, random_state=42)) ])

In [95]:
svmpipe.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf-svm',
                 SGDClassifier(alpha=0.001, loss='modified_huber', penalty='l1',
                               random_state=42))])

In [96]:
print('Training set score: ' + str(svmpipe.score(X_train,y_train)))
print('Test set score: ' + str(svmpipe.score(X_test,y_test)))

Training set score: 0.5978562103479077
Test set score: 0.6426346528228423


## PreProcess Steps

In [97]:
ca_restaurant_reviews.head(1)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,2456,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,1,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': ""...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...",bFPdtzu11Oi0f92EAcjqmg,5,What a great addition to the Funk Zone! Grab ...,2016-10-13 22:50:47


In [99]:
df=ca_restaurant_reviews[["review_stars","text"]]

In [100]:
df

,review_stars,text
0,5,What a great addition to the Funk Zone! Grab ...
1,5,Possibly the best breakfast sandwich EVER. On...
2,4,Pricey ( a ham and cheese croissant was $5.50)...
3,5,Are you kidding me? This is without a doubt so...
4,2,"Super slow service, ""salads"" are not very good..."
...,...,...
205459,5,This is one of my new favorite pizza joints he...
205460,4,Ordered vegan chicken and Daiya cheese and onl...
205461,1,This was the worst pizza I've ever had ever ea...
205462,5,One of the best pizzas in SB....seriously!!!\n...


In [102]:
df['char_count'] = df['text'].apply(lambda x: len(str(x)))
print("Average characters in 5-star reviews",df[df['review_stars']==5]['char_count'].mean())
print("Average characters in 1-star reviews",df[df['review_stars']==1]['char_count'].mean())

Average characters in 5-star reviews 421.1385576266224
Average characters in 1-star reviews 652.0448895027624


c:\Users\Rajup\anaconda3\envs\testinggrounds\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rajup\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rajup\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rajup\AppData\Roaming\nltk_data...


True

In [104]:
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [105]:
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [106]:
wl = WordNetLemmatizer()
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [107]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [126]:
df

,review_stars,text
0,5,What a great addition to the Funk Zone! Grab ...
1,5,Possibly the best breakfast sandwich EVER. On...
2,4,Pricey ( a ham and cheese croissant was $5.50)...
3,5,Are you kidding me? This is without a doubt so...
4,2,"Super slow service, ""salads"" are not very good..."
...,...,...
205459,5,This is one of my new favorite pizza joints he...
205460,4,Ordered vegan chicken and Daiya cheese and onl...
205461,1,This was the worst pizza I've ever had ever ea...
205462,5,One of the best pizzas in SB....seriously!!!\n...


In [118]:
df=df[["review_stars","text"]]

In [132]:
df.shape

(205464, 2)

In [120]:
train, test = train_test_split(df, test_size=0.4)

In [131]:
train.shape

(123278, 2)

In [130]:
test.shape

(82186, 2)

In [135]:
testing_swifter=train[["text"]]

In [136]:
### checking the newdf
len(testing_swifter)==len(train)

True

In [137]:
testing_swifter.shape

(123278, 1)

In [151]:
from swifter import swifter
from swifter import tqdm_dask_progressbar

In [158]:
import time
start_time = time.time()
testing_swifter['clean_text']= testing_swifter["text"].swifter.apply(lambda x: finalpreprocess(x))
print("--- %s seconds ---" % (time.time() - start_time))

Pandas Apply: 100%|██████████| 123278/123278 [28:56<00:00, 71.00it/s] 

--- 1772.5565288066864 seconds ---


In [160]:
len(testing_swifter)==len(train)

True

,review_stars,text
87687,1,Currently I am at Cajun Kitchen on Chapala an...


In [165]:
new_text_train=testing_swifter['clean_text'].values.tolist()
new_stars_train=train["review_stars"].values.tolist()

In [167]:
new_text_test=test['text'].values.tolist()
new_stars_test=test["review_stars"].values.tolist()

In [168]:
len(new_text_test)==len(new_stars_test)

True

In [215]:
newpipeline.fit(new_text_train,new_stars_train)

Pipeline(steps=[('vect', TfidfVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC())])

In [171]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [213]:
newpipeline = Pipeline([('vect', TfidfVectorizer()),
    ('tfidf', TfidfTransformer()), 
    ('clf', LinearSVC())])

In [216]:
print('Training set score: ' + str(newpipeline.score(new_text_train,new_stars_train)))
print('Test set score: ' + str(newpipeline.score(new_text_test,new_stars_test)))

Training set score: 0.8257515533996334
Test set score: 0.6436010999440294


In [210]:
newpipeline_nb = Pipeline([('vect', TfidfVectorizer()),
    ('tfidf', TfidfTransformer()), 
    ('nb_clf',  MultinomialNB())])

In [211]:
newpipeline_nb.fit(new_text_train,new_stars_train)

Pipeline(steps=[('vect', TfidfVectorizer()), ('tfidf', TfidfTransformer()),
                ('nb_clf', MultinomialNB())])

In [212]:
print('Training set score: ' + str(newpipeline_nb.score(new_text_train,new_stars_train)))
print('Test set score: ' + str(newpipeline_nb.score(new_text_test,new_stars_test)))

Training set score: 0.5855302649296712
Test set score: 0.5522595089187939


In [ ]:
(SGDClassifier(loss='modified_huber', penalty='l1',alpha=1e-3, random_state=42))

In [203]:
newpipeline_SGD = Pipeline([('vect', TfidfVectorizer()),
    ('tfidf', TfidfTransformer()), 
    ('SGD',SGDClassifier(loss='modified_huber', penalty='l1',alpha=1e-3, random_state=42))])

In [204]:
newpipeline_SGD.fit(new_text_train,new_stars_train)

Pipeline(steps=[('vect', TfidfVectorizer()), ('tfidf', TfidfTransformer()),
                ('SGD',
                 SGDClassifier(alpha=0.001, loss='modified_huber', penalty='l1',
                               random_state=42))])

In [195]:
testing_swifter_test=test[["text"]]

In [196]:
len(testing_swifter_test)

82186

In [197]:
testing_swifter_test['clean_text']= testing_swifter_test["text"].swifter.apply(lambda x: finalpreprocess(x))

Pandas Apply: 100%|██████████| 82186/82186 [17:33<00:00, 78.01it/s] 


In [198]:
new_text_test=testing_swifter_test['clean_text'].values.tolist()
new_stars_test=test["review_stars"].values.tolist()

In [199]:
len(new_text_test)==len(new_stars_test)

True

In [205]:
print('Test set score: ' + str(newpipeline_SGD.score(new_text_test,new_stars_test)))

Test set score: 0.5716302046577276
